In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
latent_size = 100
hidden_size = 64
image_size = 64
num_epochs = 500
batch_size = 64
lr = 0.0002
beta1 = 0.5

# Path to custom dataset folder
data_root = 'D:\projects\GANMTECH\Septoria\datasetprep\data'

# Create a directory to save generated images
os.makedirs('generated_images', exist_ok=True)

# Image processing
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
])

# Custom dataset
dataset = datasets.ImageFolder(root=data_root, transform=transform)

# DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_size, hidden_size * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(hidden_size * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 8, hidden_size * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 4, hidden_size * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 2, hidden_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Initialize generator
generator = Generator().to(device)

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))

# Training loop
total_step = len(dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
        batch_size = images.size(0)
        folder_names = [dataset.classes[label] for label in labels]

        # Train generator
        optimizer_g.zero_grad()
        noise = torch.randn(batch_size, latent_size, 1, 1, device=device)
        fake_images = generator(noise)

        # Save generated images in respective folders
        for j, folder_name in enumerate(folder_names):
            folder_path = os.path.join('generated_images', folder_name)
            os.makedirs(folder_path, exist_ok=True)
            save_image(fake_images[j], os.path.join(folder_path, f'{epoch+1}_{i*batch_size+j}.png'), normalize=True)

        optimizer_g.step()

        if (i + 1) % 200 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}]')

print("Training finished!")
